In [1]:
import numpy as np
import tensornetwork as tn
import tensorflow as tf

# Layers

In [2]:
class QuantumDMRGLayer(tf.keras.layers.Layer):
    def __init__(self, dimvec, pos_label, nblabels, bond_len):
        super(QuantumDMRGLayer, self).__init__()
        self.dimvec = dimvec
        self.pos_label = pos_label
        self.nblabels = nblabels
        self.m = bond_len

        self.mps_tensors = [tf.Variable(self.mps_tensor_initial_values(i),
                                        trainable=True,
                                        name='mps_tensors_{}'.format(i))
                            for i in range(self.dimvec)]

    def mps_tensor_initial_values(self, idx):
        if idx == 0 or idx == self.dimvec-1:
            tempmat = tf.eye(max(2, self.m))
            mat = tempmat[0:2, :] if 2 < self.m else tempmat[:, 0:self.m]
            return mat + tf.random.normal(mat.shape, mean=0.0, stddev=1e-9)
        elif idx == self.pos_label:
            return tf.random.normal((2, self.m, self.m, self.nblabels),
                                    mean=0.0,
                                    stddev=1e-9)
        else:
            return tf.random.normal((2, self.m, self.m),
                                    mean=0.0,
                                    stddev=1e-9)

    def infer_single(self, input):
        assert input.shape[0] == self.dimvec
        assert input.shape[1] == 2
        
        nodes = [
            tn.Node(self.mps_tensors[i], backend='tensorflow')
            for i in range(self.dimvec)
        ]
        input_nodes = [
            tn.Node(input[i, :], backend='tensorflow')
            for i in range(self.dimvec)
        ]

        for i in range(self.dimvec):
            nodes[i][0] ^ input_nodes[i][0]
        nodes[0][1] ^ nodes[1][1]
        for i in range(1, self.dimvec-1):
            nodes[i][2] ^ nodes[i+1][1]

        final_node = tn.contractors.auto(nodes + input_nodes,
                                         output_edge_order=[nodes[self.pos_label][3]])
        return final_node.tensor

    def call(self, inputs):
        return tf.vectorized_map(self.infer_single, inputs)


In [3]:
quantum_dmrg_model = tf.keras.Sequential([
    tf.keras.Input(shape=(748, 2)),
    QuantumDMRGLayer(dimvec=748, pos_label=5, nblabels=10, bond_len=5),
    # tf.keras.layers.Softmax()
])
quantum_dmrg_model.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy())

In [4]:
quantum_dmrg_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
quantum_dmrg_layer (QuantumD (None, 10)                37770     
Total params: 37,770
Trainable params: 37,770
Non-trainable params: 0
_________________________________________________________________


In [5]:
cosx = np.random.uniform(size=748)

inputs = np.array([np.array([np.cos(0.5*np.pi*cosx/256.), np.sin(0.5*np.pi*cosx/256.)]).T])

In [6]:
inputs

array([[[0.99999675, 0.00255093],
        [0.99998501, 0.00547526],
        [0.99999381, 0.00351938],
        ...,
        [0.99998196, 0.00600701],
        [0.99998354, 0.00573764],
        [0.99999875, 0.00158424]]])

In [7]:
output = quantum_dmrg_model.predict(inputs)
output

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [8]:
[quantum_dmrg_model.layers[0].mps_tensors[i].shape for i in range(10)]

[TensorShape([2, 5]),
 TensorShape([2, 5, 5]),
 TensorShape([2, 5, 5]),
 TensorShape([2, 5, 5]),
 TensorShape([2, 5, 5]),
 TensorShape([2, 5, 5, 10]),
 TensorShape([2, 5, 5]),
 TensorShape([2, 5, 5]),
 TensorShape([2, 5, 5]),
 TensorShape([2, 5, 5])]